Import

In [4]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [13]:
soup = ""
with open("data.xml", "r", encoding="utf-8") as file:
    content = file.read()
    soup = BeautifulSoup(content, "xml")
    pretty_xml = soup.prettify()
with open('data.xml', 'w', encoding="utf-8") as file:
    file.write(pretty_xml)
    file.close()

